<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-20 15:03:10
-------------------
qualified stocks: 96
with latest results: 33
still star stocks: 19
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  55.89 K
Current:  1.27 C
-------------------
Today PnL: -1.35 L (-1.05%)
Current PnL: -16.74 L (-11.87%)
CY Booked + Current PnL: -13.58 L (-9.63%)
-------------------
Total profit:  4.25 L
Total loss:  -20.99 L
-------------------
Total Booked + Current PnL: 13.40 L (11.75%)
Total Booked PnL: 30.14 L (26.43%)
Curr Year Booked PnL: 3.16 L (2.49%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 79.84 L (62.81%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.41 C (111.21%)
Deployed:  1.14 C
Current:  1.27 C
CAGR/XIRR %: 7.76%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SBILIFE,1496.49,1924.99,-0.61,H-LC,71.88,147580.0,21875.0,14123.0,-0.74,17.40,9.57,28.63,159.0,1.55,1.17,31.47,AR,ATH,INSURANCE
63,RELIANCE,1288.53,1526.00,4.61,X-LC,79.17,225513.0,21925.0,15605.0,-0.94,10.77,6.92,18.43,161.0,1.40,1.78,22.84,XY25,NTT,REFINERIES
6,ANGELONE,2328.67,3033.00,10.77,H-SC,90.62,180895.0,29531.0,16244.0,-0.69,19.51,8.98,30.25,149.0,1.82,1.43,43.17,X40N,NTT,FINANCE
57,PIDILITIND,2504.06,3576.00,-18.67,X-LC,32.81,90102.0,14980.0,17182.0,-2.11,19.94,19.07,42.81,121.0,0.87,0.71,13.00,X40,BTT,CHEMICALS
71,SBIN,760.30,863.00,-14.39,M-LC,20.83,210675.0,9196.0,18013.0,0.37,4.56,8.55,13.51,60.0,0.51,1.66,17.74,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PFIZER,4236.33,6318.30,-1.18,H-SC,97.92,153884.0,22558.0,41980.0,11.10,17.18,27.28,49.15,94.0,0.54,1.22,26.83,X40,ATH,PHARMA
21,COFFEEDAY,59.14,80.00,-57.75,L-SC,92.71,61574.0,-51975.0,92022.0,4.97,-45.77,149.45,35.27,181.0,-0.56,0.49,50.70,XR,NTT,HOTELS
86,TTKPRESTIG,769.29,1149.24,-1.47,X-SC,57.29,89997.0,-10780.0,60550.0,2.32,-10.70,67.28,49.39,172.0,-0.18,0.71,17.12,X40N,ATH,DURABLES
30,GREENPANEL,375.16,537.00,176.49,M-SC,61.46,123464.0,-59614.0,138588.0,2.21,-32.56,112.25,43.14,109.0,-0.43,0.98,16.13,XY24,NTT,MISC
28,FINEORG,4771.46,7168.71,-17.33,H-SC,68.23,78809.0,-2306.0,43061.0,2.03,-2.84,54.64,50.24,129.0,-0.05,0.62,32.33,X40N,ATH,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,QUESS,711.66,986.0,NaN,NaN,22.92,74318.0,-81536.0,141620.0,-9.08,-52.32,190.56,38.55,12.0,-0.58,0.59,13.80,XY24,NTT,MISC
84,TITAGARH,1097.23,1548.0,-11.82,H-SC,88.54,154936.0,-37079.0,115970.0,-5.51,-19.31,74.85,41.08,119.0,-0.32,1.22,31.13,XY24,NTT,ENGINEERING
92,VIPIND,488.80,718.0,-840.11,H-SC,100.00,70483.0,-24833.0,69524.0,-4.68,-26.05,98.64,46.89,173.0,-0.36,0.56,39.02,X40N,NTT,MISC
27,EASEMYTRIP,18.12,26.4,16.95,M-SC,6.25,112607.0,-64208.0,145004.0,-4.55,-36.31,128.77,45.70,22.0,-0.44,0.89,4.53,XY24,NTT,TRAVEL
11,BANDHANBNK,214.25,400.0,-35.72,H-SC,21.88,191400.0,-57130.0,272592.0,-3.25,-22.99,142.42,86.70,51.0,-0.21,1.51,26.62,XY24,NTT,BANKS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIAMART,2311.97,4911.36,-51.65,H-SC,58.33,104456.0,2729.0,111643.0,-0.39,2.68,106.88,112.43,1.0,0.02,0.83,23.28,AR,ATH,MISC
83,TCS,3928.29,4998.00,-16.16,X-LC,59.38,189054.0,-23074.0,80839.0,-0.51,-10.88,42.76,27.23,3.0,-0.29,1.49,8.34,X40,BTT,IT
82,TATAMOTORS,776.02,1065.00,-52.64,M-LC,91.67,190363.0,-15282.0,91869.0,-1.53,-7.43,48.26,37.24,4.0,-0.17,1.50,23.91,XY24,NTT,AUTO
10,AWL,342.88,485.00,-69.05,H-MC,4.17,167830.0,-49899.0,140138.0,-1.77,-22.92,83.50,41.45,4.0,-0.36,1.33,12.11,XY24,NTT,FMCG
42,INFY,1432.81,2275.00,-12.78,X-LC,66.67,216812.0,17651.0,99408.0,-1.89,8.86,45.85,58.78,6.0,0.18,1.71,13.20,X40,BTT,IT


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,POLYCAB,5005.92,7541.35,7.69,H-MC,84.38,168462.0,28296.0,42688.0,-1.41,20.19,25.34,50.65,56.0,0.66,1.33,28.57,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-2.22,M-LC,85.42,189498.0,38292.0,27364.0,-1.08,25.32,14.44,43.43,48.0,1.40,1.50,31.35,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LUXIND,1448.19,2442.00,43.68,M-SC,69.79,149084.0,11506.0,82906.0,-1.54,8.36,55.61,68.62,66.0,0.14,1.18,29.25,X40N,NTT,TEXTILES
6,ANGELONE,2328.67,3033.00,10.77,H-SC,90.62,180895.0,29531.0,16244.0,-0.69,19.51,8.98,30.25,149.0,1.82,1.43,43.17,X40N,NTT,FINANCE
20,COALINDIA,406.65,512.15,8.40,L-LC,45.83,151182.0,722.0,38310.0,1.47,0.48,25.34,25.94,85.0,0.02,1.19,15.36,XY25,ATH,MINING
58,POLYCAB,5005.92,7541.35,7.69,H-MC,84.38,168462.0,28296.0,42688.0,-1.41,20.19,25.34,50.65,56.0,0.66,1.33,28.57,X40N,ATH,CABLES
29,GILLETTE,8109.44,10330.69,0.60,H-SC,83.33,148410.0,10550.0,27218.0,1.17,7.65,18.34,27.39,64.0,0.39,1.17,29.91,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-3.78,H-SC,16.67,108122.0,6118.0,38902.0,-2.28,6.00,35.98,44.14,31.0,0.16,0.85,23.21,XR,ATH,MISC
95,WIPRO,243.46,420.00,-9.71,M-LC,56.25,154826.0,3881.0,105576.0,-0.81,2.57,68.19,72.51,49.0,0.04,1.22,16.57,XR,NTT,IT
12,BANKINDIA,113.49,190.00,-33.09,M-MC,17.71,131951.0,2799.0,84264.0,-1.69,2.17,63.86,67.42,57.0,0.03,1.04,28.33,XR,NTT,BANKS
39,INDIAMART,2311.97,4911.36,-51.65,H-SC,58.33,104456.0,2729.0,111643.0,-0.39,2.68,106.88,112.43,1.0,0.02,0.83,23.28,AR,ATH,MISC
1,AARTIIND,487.04,919.00,37.28,M-SC,87.50,78799.0,-2537.0,74678.0,-3.06,-3.12,94.77,88.69,169.0,-0.03,0.62,32.88,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VALIANTORG,512.64,838.00,-343.92,H-SC,73.96,103727.0,-29559.0,114152.0,1.41,-22.18,110.05,63.47,145.0,-0.26,0.82,73.87,XR,NTT,CHEMICALS
87,UJJIVANSFB,52.77,60.00,38.86,M-SC,39.58,120798.0,-21681.0,41204.0,-0.45,-15.22,34.11,13.70,213.0,-0.53,0.95,42.57,X40N,NTT,BANKS
12,BANKINDIA,113.49,190.00,-33.09,M-MC,17.71,131951.0,2799.0,84264.0,-1.69,2.17,63.86,67.42,57.0,0.03,1.04,28.33,XR,NTT,BANKS
70,SBILIFE,1496.49,1924.99,-0.61,H-LC,71.88,147580.0,21875.0,14123.0,-0.74,17.40,9.57,28.63,159.0,1.55,1.17,31.47,AR,ATH,INSURANCE
88,UNIONBANK,123.87,163.00,-9.51,M-LC,65.62,157179.0,16339.0,28151.0,0.93,11.60,17.91,31.59,70.0,0.58,1.24,36.53,XY24,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,718.00,-840.11,H-SC,100.00,70483.0,-24833.0,69524.0,-4.68,-26.05,98.64,46.89,173.0,-0.36,0.56,39.02,X40N,NTT,MISC
94,WHIRLPOOL,1151.42,2270.00,-34.66,M-SC,98.96,164490.0,18260.0,123795.0,0.12,12.49,75.26,97.15,28.0,0.15,1.30,40.52,X40,NTT,DURABLES
56,PFIZER,4236.33,6318.30,-1.18,H-SC,97.92,153884.0,22558.0,41980.0,11.10,17.18,27.28,49.15,94.0,0.54,1.22,26.83,X40,ATH,PHARMA
51,LTIM,5664.39,7262.39,-6.27,X-LC,96.88,150303.0,-19629.0,67561.0,-0.20,-11.55,44.95,28.21,32.0,-0.29,1.19,23.49,X200,ATH,IT
80,TANLA,917.30,1963.11,-39.81,H-SC,95.83,147060.0,-92355.0,365312.0,-3.05,-38.58,248.41,114.01,20.0,-0.25,1.16,34.88,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.02
1,25,38.32
2,50,67.67


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.40
X40N    15.26
X40     14.32
AR      13.95
XR      10.37
XY25     9.39
X200     1.19
SR       1.10
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.65
M-SC    17.66
X-LC    15.93
H-MC     8.85
M-LC     8.27
H-LC     5.90
M-MC     5.52
X-MC     5.51
X-SC     2.81
L-SC     2.29
L-LC     1.19
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.52,-1.84,34.23
BANKS,10.87,-9.36,62.99
IT,9.04,-20.76,88.13
FINANCE,7.46,-17.30,70.24
MISC,7.09,-26.44,80.84
INSURANCE,6.12,0.24,29.83
PAINTS,5.51,-17.99,52.91
AUTO,4.28,-11.54,60.15
ELECTRICAL,4.06,-8.33,63.53


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2998884.0,27
AR,1298620.0,15
XR,1231223.0,15
X40N,1230176.0,18
X40,781619.0,12
SR,206197.0,2
XY25,169419.0,6
X200,67561.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2321805.0,22
M-SC,2001324.0,21
X-LC,725141.0,12
M-MC,569521.0,5
H-MC,488390.0,8
X-SC,353205.0,4
M-LC,332548.0,6
X-MC,308049.0,5
L-SC,307584.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      906676.0      6
M-SC       XY24      758627.0      6
H-SC       AR        536456.0      3
           X40N      527656.0      7
M-SC       X40N      443088.0      6
           XR        424990.0      5
M-MC       XY24      400965.0      3
X-LC       X40       387737.0      5
H-SC       XR        281819.0      4
M-LC       XY24      208959.0      4
M-SC       X40       196492.0      2
H-LC       AR        196233.0      4
X-LC       AR        186510.0      3
H-MC       XY24      180161.0      2
L-SC       XR        157620.0      2
X-MC       XY24      145641.0      2
H-MC       AR        138323.0      2
X-SC       SR        120941.0      1
           XR        118964.0      1
           X40N      113300.0      2
M-LC       XR        105576.0      1
X-MC       X40N      103444.0      2
M-SC       AR         92871.0      1
L-SC       XY24       86029.0      1
M-SC       SR         85256.0      1
M-MC       AR         84292.0      1
           XR         84264.0      1
X-LC       XY25       83333.0      3
H-MC       X40        69228.0      2
H-SC       X40        69198.0      2
X-LC       X200       67561.0      1
L-SC       AR         63935.0      1
X-MC       X40        58964.0      1
H-MC       XR         57990.0      1
           X40N       42688.0      1
L-LC       XY25       38310.0      1
H-LC       XY25       29763.0      1
M-LC       XY25       18013.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
